### Step 1 – Load the Dataset


In [29]:
import pandas as pd
area_fire= pd.read_csv(r'CA (cleaned+unit id) wildfire area data (1984 to 2023) .csv')
weather_fire= pd.read_csv(r'CA_Weather (cleaned up) 1984-2023.csv')

: 

In [ ]:
area_fire.head()

#check the number of features and amount of entries
print('no. of data, no. of features:', area_fire.shape)
#check the value and type of each column(features)
print('data info:', area_fire.info())


Cleaning up the data, dropping extra column


In [18]:
area_fire=area_fire.drop(['CONT_DATE'],axis=1)


In [21]:
area_date=area_fire.iloc[:, 0:4]
area_date.set_index(['ALARM_DATE'])
weather_fire.set_index(['DATE'])


,FIRE_START_DAY,PRECIPITATION,MAX_TEMP,MIN_TEMP,AVG_WIND_SPEED,TEMP_RANGE,WIND_TEMP_RATIO,LAGGED_PRECIPITATION,LAGGED_AVG_WIND_SPEED,SEASON,DAY_OF_YEAR
DATE,,,,,,,,,,,
31/12/2023,False,0.00,60,51,4.92,9,0.082000,0.62,5.558571,Winter,365
30/12/2023,False,0.62,62,54,8.50,8,0.137097,0.62,5.558571,Winter,364
29/12/2023,False,0.00,65,52,3.80,13,0.058462,0.00,5.175714,Winter,363
28/12/2023,False,0.00,64,50,5.59,14,0.087344,0.50,5.782857,Winter,362
27/12/2023,False,0.00,62,52,5.59,10,0.090161,1.83,6.198571,Winter,361
...,...,...,...,...,...,...,...,...,...,...,...
5/1/1984,False,0.00,74,49,5.14,25,0.069459,0.00,5.100000,Winter,5
4/1/1984,False,0.00,76,45,4.70,31,0.061842,0.00,5.090000,Winter,4
3/1/1984,False,0.00,70,47,5.37,23,0.076714,0.00,5.220000,Winter,3


### Step 2 – Preprocess the Dataset
 
 The features are all on different scales. To bring them all to a common scale, we’ll use the **StandardScaler** that transforms the features to have zero mean and unit variance:

##### Thoughts
run a PCA to see which top 2-3 factors are the most crucial?

In [12]:
import sklearn 
from sklearn.preprocessing import StandardScaler
#separate features and
features = ['sepal length', 'sepal width', 'petal length', 'petal width']
std_scaler = StandardScaler()
scaled_cafire = std_scaler.fit_transform(ca_fire)

ValueError: could not convert string to float: '1984-01-01'

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split